In [19]:
# Beau Schwieso
# I decided not to implement the interpreter function like suggested. 


## Programming Assignment 2 - Part 1 and 2 
### Cpts 355 - Spring 2016
### An Interpreter for a Postscript-like Language

### Assigned Feb. 3, 2016

### Due Friday, Feb. 12, 2016
Develop your code in a file named `sps.ipynb`, starting from this notebook file. When you are finished, upload `sps.ipynb` on the course Turnin Page. 

The entire interpreter project (Parts 1 and Part 2 together) will count for 10% of your course grade. This first part is worth 20% of that 10%: the intention is to make sure that you are on the right track and have a chance for mid-course correction before completing Part 2. However, note that the work and amount of code involved in Part 1 is a large fraction of the total project, so you need to get going on this part right away.

### This assignment is to be your own work. Refer to the course academic integrity statement in the syllabus.

## The problem
In this assignment you will write an interpreter in Python for a small PostScript-like language, concentrating on key computational features of the abstract machine, omitting all PS features related to graphics, and using a somewhat-simplified syntax.

The simplified language, SPS, has the following features of PS
* integer constants, e.g. `123`: in python3 there is no practical limit on the size of integers
* boolean constants, `true` and `false` (Note that the boolean constants in python are `True` and `False`)
* name constants, e.g. `/fact`: start with a `/` and letter followed by an arbitrary sequence of letters and numbers
* names to be looked up in the dictionary stack, e.g. `fact`: as for name constants, without the `/`
* code constants: code between matched curly braces `{` ... `}`
* built-in operators on numbers: `add`, `sub`, `mul`, `div`, `eq`, `lt`, `gt`
* built-in operators on boolean values: `and`, `or`, `not`; these take boolean operands only. Anything else is an error.
* built-in sequencing operators: `if`, `ifelse`; make sure that you understand the order of the operands on the stack. Play with ghostscript if necessary to help understand what is happening.
* stack operators: `dup`, `exch`, `pop`
* dictionary creation operator: `dict`; takes one operand from the operand stack, ignores it, and creates a new, empty dictionary on the operand stack
* dictionary stack manipulation operators: `begin`, `end`. `begin` requires one dictionary operand on the operand stack; `end` has no operands.
* name definition operator: `def`. This requires two operands, a name and a value
* defining (using `def`) and calling functions
* stack printing operator (prints contents of stack without changing it): `stack`
* top-of-stack printing operator (pops the top element of the stack and prints it): `=`


## Requirements for Part 1 (Due Feb. 12)
In Part 1 you will build some essential pieces of the interpreter but not yet the full interpreter. The pieces you build will be driven by Python test code rather than actual Postscript programs. The pieces you are going to build first are:
* The operand stack
* The dictionary stack
* The operators that don't involve code arrays: all of the operators except `if`, `ifelse`.
In Part 2 we will add the implementations for `if`, `ifelse`, calling functions, as well as interpreting input strings in the Postscript language.
* Looking up names

### The operand stack
The operand stack should be implemented as a Python list. The list will contain **Python** integers, booleans, and strings, and later in Part 2 code arrays. Python integers and booleans on the stack represent Postscript integers and booleans. Python strings on the stack represent names of Postscript variables (see the handling of names and the `def` operator below.

When using a list as a stack one of the decisions you have to make is where the *hot* end of the stack is located. (The *hot* end is where pushing and popping happens). Will the hot end be at position `0`, the head of the list, or at position `-1`, the end of the list? It's your choice.

### The dictionary stack
The dictionary stack is also implemented as a Python list. It will contain **Python** dictionaries which will be the implementation for **Postscript** dictionaries. The dictionary stack needs to support adding and removing dictionaries at the hot end, as well as defining and looking up names. 

### Operators
Operators will be implemented as zero-argument Python functions that manipulate the operand and dictionary stacks. For example, the `add` operator could be implemented as the Python function (with comments instead of actual implementations)
```
def add():
    op1 = # pop the top value off the operand stack
    op2 = # pop the top value off the operand stack
    # push (op1 + op2) onto the operand stack
```
You may run into conflicts for some of the names of these functions . For example, the function for the `not` operator can't be named `not` because it is reserved for another use in Python. So you could do something like:
```
def psnot():
    // pop the top value off the operand stack and push its negation onto the operand stack
```    
The `begin` and `end` operators are a little different in that they manipulate the dictionary stack in addition to or instead of the operand stack. Remember that the `dict` operator affects *only* the operand stack.

The `def` operator takes two operands from the operand stack: a string (recall that strings in the operand stack represent names of postscript variables) and a value. It changes the dictionary at the hot end of the dictionary stack so that the string is mapped to the value by that dictionary. Notice that `def` does ***not*** change the number of dictionaries on the dictionary stack!

### Name lookup

Name lookup is implemented by a Python function:
```
def lookup(name):
    # search the dictionaries on the dictionary stack starting at the hot end to find one that contains name
    # return the value associated with name
```
Note that name lookup is ***not*** a Postscript operator, but it ***is*** implemented by a Python function.

## Your Code Start Here

In [20]:
# The operand stack: define the operand stack and its operations in this notebook cell
opstack = []
# other globals for me to use



# now define functions to push and pop values on the opstack according to your decision about which
# end should be the hot end. Recall that `pass` in python is a no-op: replace it with your code.

def opPop(): 
    return opstack.pop()

def opPush(value):
    print("in opPush %s" % value)
    opstack.append(value)

# Remember that there is a Postscript operator called "pop" so we choose different names for these functions.


In [21]:
# The dictionary stack: define the dictionary stack and its operations in this cell
dstack = [{}]

# now define functions to push and pop dictionaries on the dictstack, to define name, and to lookup a name

def dictPop():
    a = dstack.pop()
    if(a == {}):
        dictPush(a)
        return {}
    else:
        return a
    

def dictPush(value):
    dstack.append(value)

def define(name, value):
    a = opstack.pop()
    b = opstack.pop()
    x = dstack.pop()
    x[b] = a
    dstack.append(x)

def lookup(name):
    if type(name) == type(1):    
        return name    
    else:       
        for a in dstack:
            if(a == {}):
                return None            
            return a[name] #returns None if not found
        

In [22]:
# Arithmetic operators: define all the arithmetic operators in this cell -- add, sub, mul, div, eq, lt, gt

def Equal():
    lookup(opstack.pop())


def Add():
    op2 = opPop()
    op1 = opPop()
    opPush(op1 + op2)
    
def Sub():
    op2 = opPop()
    op1 = opPop()
    opPush(op1 - op2)

def Mul():
    op2 = opPop()
    op1 = opPop()
    opPush(op1 * op2)
    
def Div():
    op2 = opPop()
    op1 = opPop()
    opPush(op1 / op2)
    
def EQ():
    op2 = opPop()
    op1 = opPop()
    opPush(op1 == op2)

    

def Lt():
    op2 = opPop()
    op1 = opPop()
    opPush(op1 < op2)

def Gt():
    op2 = opPop()
    op1 = opPop()
    opPush(op1 > op2)


In [23]:
# Boolean operators: define all the boolean operators in this cell -- and, or, not
def And():
    boolVal1 = opPop()
    boolVal2 = opPop()
    opPush (boolVal1 and boolVal2)

def myOr():
    boolVal1 = opPop()
    boolVal2 = opPop()
    opPush (boolVal1 or boolVal2)

def Not():
    boolVal1 = opPop()
    opPush (not boolVal1)

In [24]:
# Define the stack manipulation operators in this cell: dup, exch, pop
def Dup():
    a = opstack.pop()
    opstack.append(a)
    opstack.append(a)
    
def Exch():
    a = opstack.pop()
    b = opstack.pop()
    
    opstack.append(a)
    opstack.append(b)
def Pop():
    opstack.pop()

In [25]:
# Define the dictionary manipulation operators in this cell: dict, begin, end, def
# name the function for the def operator psDef because def is reserved in Python
def Dict():
    # takes one operand from the operand stack, ignores it, and creates a new, empty dictionary on the operand stack
    opstack.pop()
    dstack.append({})
    
def Begin(d):
    # begin pushes a dictionary on the dictionary stack
    dstack.append(d)
def End():
    #  end removes a dictionary from the dictionary stack
    dstack.pop()
    
def Def():
    # def creates or modifies an entry in the top dictionary on the dictionary stack.
    a = opstack.pop()
    b = opstack.pop()
    d = dstack.pop()
    d[str(b)] = a
    dstack.append(d)

In [26]:
# Define the IF ELSE and ELSEIF operators
# ELSE will be handled in processLine

def If():
    #print ("in IF before pop")
    stack()
    l = opstack.pop()
    b = opstack.pop()
    if b:
        print('was true : %s ' % l[1:-1])
        processLine(l[1:-1])
        
        
def ElseIf():
    lf = opstack.pop()
    lt = opstack.pop()
    b = opstack.pop()
    
    if b == 'true':
        processLine(lt[1:-1])
    else:
        processLine(lf[1:-1])
        
    
    


In [27]:
# Define processLine
def processLine(line):
    var = ''
    brace = ''
    curly = False
    
    for c in line.split():
        if c == '{': # begin of code array
            brace = c
            curly = True
        elif curly:
            brace +=' '
            brace += c
            if  c == '}':
                curly = False
                opstack.append(brace) # should be finishing up the final piece of code array here
        elif c[0] == '/':
            var = c[1:]
            opstack.append(var)
        elif c[0] >='0' and c[0] <= '9': #if a number
            opstack.append(int(c))
        else:
            handleTheRest(c)

In [28]:
# Define the printing operators in this cell: stack, etc

def printDstack():
    if not dstack:
        print ("\nEMPTY DICT STACK\n")
        return
    print ('******** T O P ********')
    for element in reversed (dstack):
        print (element)
    print ('***** B O T T O M *****')
    
def stack():
    if not opstack:
        print ("\nEMPTY OPSTACK\n")
        return
    print ('******** T O P ********')
    for element in reversed (opstack):
        print (element)
    print ('***** B O T T O M *****')
    
def PrintVal():
    a = lookup(opstack.pop())
    print (a)

In [29]:
def handleTheRest(token):
    if token == '=': Equal()
    elif token == 'and': And()
    elif token == 'or' : Or()
    elif token == 'eq' : EQ()
    elif token == 'not': Not()
    elif token == 'gt' : Gt()
    elif token == 'lt' : Lt()
    elif token == 'add': Add()
    elif token == 'sub': Sub()
    elif token == 'mul': Mul()
    elif token == 'div': Div()
    elif token == 'stack': PrintOP()
    elif token == 'if': If()
    elif token == 'ifelse': ElseIf()
    elif token == 'begin': Begin()
    elif token == 'dup': Dup()
    elif token == 'exch': Exch()
    elif token == 'pop': Pop()
    elif token == 'def': Def()
    elif token == 'end': End()
        
    else:
        s = lookup(str(token))
        if type(s) is not int:
            if(s[0] == '{'):
                processLine(s[1:-1])
        else:
            opstack.append(s)

## Test your code
With all of that stuff defined, you will be able to test your interpreter using Python code like this:

In [33]:
opstack[:] = [] # clear the stack before our runs

gs1 = "/x 3 def"
gs2 = "/y 4 def"
gs3 = "x 4 lt"
stack() # WHY IS THIS EMPTY THOUGH?! UGH
gs4 = "{ x 7 add } if"
GS = [gs1, gs2, gs3, gs4]

for x in GS:
    processLine(x)
    

stack()
#printDstack()


EMPTY OPSTACK

in opPush True
******** T O P ********
{ x 7 add }
True
***** B O T T O M *****
was true :  x 7 add  
in opPush 10
******** T O P ********
10
***** B O T T O M *****
******** T O P ********
{'x': 3, 'y': 4}
***** B O T T O M *****


## Comment:
- Correct Algorithm!
- Good Coding Style!
- Excellent Overall!
- Good to see your own implementation. But remember it is not always read in one line of a real interpreter. For example in Python's interpreter. Users can input multi-lines' command via backslash "\":
```
Python 3.5.1 (default, Dec  7 2015, 21:59:10)
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.1.76)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 3 + 5 \
...   + 6
14
>>>
```
## Final Score = 100
